<a href="https://colab.research.google.com/github/olonok69/LLM_Notebooks/blob/main/langchain/Langchain_Document_classification_Chromadb_Sentence_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [110]:

!pip install -Uqqq pip --progress-bar off
!pip install -qqq torch==2.1 --progress-bar off
!pip install -qqq transformers==4.34.0 --progress-bar off
!pip install -qqq accelerate==0.23.0 --progress-bar off
!pip install -qqq bitsandbytes==0.41.1 --progress-bar off

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 2.16.1 requires huggingface-hub>=0.19.4, but you have huggingface-hub 0.17.3 which is incompatible.


In [111]:
!pip install sentence-transformers spacy langchain trl datasets pypdf jq html2text  chromadb unstructured -q

# Chromadb

Chroma is the open-source embedding database. Chroma makes it easy to build LLM apps by making knowledge, facts, and skills pluggable for LLMs.

https://docs.trychroma.com/

https://python.langchain.com/docs/integrations/vectorstores/chroma


In [112]:
# Import libraries
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer, AutoModelForCausalLM
import torch
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import transformers
device = 'cuda' if torch.cuda.is_available() else 'cpu'
import chromadb
import pandas as pd
from chromadb.utils import embedding_functions



In [113]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [114]:
data_path = "/content/drive/MyDrive/data/doc_classification"

In [115]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import DirectoryLoader

#embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L12-v2")

In [116]:
model_name='sentence-transformers/gtr-t5-large'#'google/flan-t5-large'

model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of T5ForConditionalGeneration were not initialized from the model checkpoint at sentence-transformers/gtr-t5-large and are newly initialized: ['decoder.block.0.layer.0.SelfAttention.k.weight', 'decoder.block.0.layer.0.SelfAttention.o.weight', 'decoder.block.0.layer.0.SelfAttention.q.weight', 'decoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight', 'decoder.block.0.layer.0.SelfAttention.v.weight', 'decoder.block.0.layer.0.layer_norm.weight', 'decoder.block.0.layer.1.EncDecAttention.k.weight', 'decoder.block.0.layer.1.EncDecAttention.o.weight', 'decoder.block.0.layer.1.EncDecAttention.q.weight', 'decoder.block.0.layer.1.EncDecAttention.v.weight', 'decoder.block.0.layer.1.layer_norm.weight', 'decoder.block.0.layer.2.DenseReluDense.wi.weight', 'decoder.block.0.layer.2.DenseReluDense.wo.weight', 'decoder.block.0.layer.2.layer_norm.weight', 'decoder.block.1.layer.0.SelfAttention.k.weight', 'decoder.block.1.layer.0.SelfAttention.o.weight', 'decoder.block.1.layer.0.

# Langchain DirectoryLoader

https://python.langchain.com/docs/modules/data_connection/document_loaders/file_directory

In [117]:
txt_loader = DirectoryLoader(path = data_path , glob = '*.txt', recursive=True)
docs = txt_loader.load()

In [103]:
len(docs)

1000

In [118]:
docs[0]

Document(page_content='The Arizona Attorney General (AG) Mark Brnovich has filed a consumer fraud lawsuit against Google, alleging that the company used “deceptive” practices to track the location of users even after they turned off location tracking.\n\nBrnovich shared information about the lawsuit on his Twitter account. He accused Google of using “deceptive and unfair practices to obtain users’ location data”. This data is exploited for advertising, which accounts for more than 80 per cent of Google’s revenue.\n\nGoogle collects detailed information about its users, including their physical locations, to target users for advertising. Often, this is done without the users’ consent or knowledge.\n\nBrnovich wrote on Twitter: “Google collects detailed information about its users, including their physical locations, to target users for advertising. Often, this is done without the users’ consent or knowledge.”\n\nHe told the Washington Post that Google has been trying to find “misleading

In [119]:
# Text Summarization Pipeline
text_summarization_pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="summarization",
    temperature=0.2,
    repetition_penalty=1.1,
      max_new_tokens=2000,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,


)
llm = HuggingFacePipeline(pipeline=text_summarization_pipeline)

In [120]:
device

'cuda'

In [121]:
for doc in docs:
    num_tokens = llm.get_num_tokens(doc.page_content)
    print (f"file{doc.metadata['source']} has {num_tokens} tokens")

file/content/drive/MyDrive/data/doc_classification/technologie/technologie_1.txt has 768 tokens
file/content/drive/MyDrive/data/doc_classification/technologie/technologie_10.txt has 607 tokens
file/content/drive/MyDrive/data/doc_classification/technologie/technologie_100.txt has 789 tokens
file/content/drive/MyDrive/data/doc_classification/technologie/technologie_11.txt has 694 tokens
file/content/drive/MyDrive/data/doc_classification/technologie/technologie_12.txt has 305 tokens
file/content/drive/MyDrive/data/doc_classification/technologie/technologie_13.txt has 778 tokens
file/content/drive/MyDrive/data/doc_classification/technologie/technologie_14.txt has 974 tokens
file/content/drive/MyDrive/data/doc_classification/technologie/technologie_15.txt has 346 tokens
file/content/drive/MyDrive/data/doc_classification/technologie/technologie_16.txt has 858 tokens
file/content/drive/MyDrive/data/doc_classification/technologie/technologie_17.txt has 638 tokens
file/content/drive/MyDrive/dat

In [122]:
import pandas as pd
from chromadb.utils import embedding_functions
from langchain.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

In [123]:
purview_classes_path = "/content/drive/MyDrive/data/Microsoft_Purview_Classifiers_with_ddg_to_load.csv"

# Sentence Transformers

https://huggingface.co/sentence-transformers

# sentence-transformers/gtr-t5-large

https://huggingface.co/sentence-transformers/gtr-t5-large

In [63]:
#chroma_client.delete_collection(name="purview_data")

In [124]:


chroma_path = "/content/drive/MyDrive/chroma_persistence"
#delete collection if this exists



model_name = "sentence-transformers/gtr-t5-large"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)


#creating a new collection , cosine similarity it will be the metrics to measure similitude
#collection = chroma_client.create_collection(name="purview_data")#, metadata={"hnsw:space": "cosine"})

In [127]:
from langchain.document_loaders import CSVLoader

# Load data from a CSV file using CSVLoader
loader = CSVLoader(purview_classes_path, metadata_columns=["classifier"])
documents = loader.load()


In [131]:
documents[0].page_content


'detail: Gory images show blood/gore. Use the API You can detect adult content with the Analyze Image 3.2 API. When you add the value of Adult to the visualFeatures query parameter, the API returns three boolean properties— isAdultContent, isRacyContent, and isGoryContent —in its JSON response. Computer Vision provides a pre-trained model that can detect adult or racy content in images and videos and give you a score and labels. Table of Contents Question Answer Explanation Question You want to build a solution that needs to detect images that contain adult content or depict violent, gory scenes. Cloud Vision API can automatically identify and flag explicit or inappropriate content within an image using five categories: adult, spoof, medical, violence, and racy. The API provides a score that indicates the likelihood for each category in the image, which you can use to set thresholds in your application and decide how to handle those ... Adult, racy, and gory images Note Images must be 

In [133]:
vectordb = Chroma.from_documents(documents=documents,

                                 # Chose the embedding you want to use
                                 embedding=hf,
                                 collection_metadata = {"hnsw:space": "cosine"},
                                 persist_directory=chroma_path)

In [134]:
vectordb.persist()
vectordb = None

In [135]:
vectordb = Chroma(persist_directory=chroma_path,
                  embedding_function=hf

                  )

The similarity_search_with_score function in LangChain with Chroma DB returns higher scores for less relevant documents because it uses cosine distance as the scoring metric. In cosine distance, a lower score indicates a higher similarity between the query and the document. Therefore, documents with lower scores are more relevant to the query.

In [136]:
res = vectordb.similarity_search(docs[100].page_content, k=5)

In [137]:
for doc in res:
    print("-" * 80)
    print(doc.page_content)
    print("-" * 80)
    print(f"Class {doc.metadata.get('classifier')}")
    print("-" * 80)

--------------------------------------------------------------------------------
detail: Data Protection Industrial & Corporate Espionage: What Is It, Cases & Best Prevention Practices March 24, 2023 Knowledge is power. Especially in the hands of your competitors. Information about your company, its products and services, finances, sales, and marketing strategy is a weapon of modern economic warfare. Employees in almost every industry today use messaging platforms like Slack, Microsoft Teams, and Signal to conduct business. But on many of those platforms, messages are "ephemeral": messages can ... If you suspect that you are being sabotaged at work, it's important to recognize the signs early on and take appropriate action to protect yourself and your professional reputation. In this blog, we will explore common signs of workplace sabotage and provide insights on how to create a mindset to navigate these challenging situations. murky world of corporate sabotage and espionage. - By Mich

In [138]:
docs_with_score = vectordb.similarity_search_with_score(docs[100].page_content, k=5)

In [139]:
for doc, score in docs_with_score:
    print("-" * 80)
    print("Score: ", score)
    print(doc.page_content)
    print("-" * 80)
    print(f"Class {doc.metadata.get('classifier')}")
    print("-" * 80)

--------------------------------------------------------------------------------
Score:  0.39295029640197754
detail: Data Protection Industrial & Corporate Espionage: What Is It, Cases & Best Prevention Practices March 24, 2023 Knowledge is power. Especially in the hands of your competitors. Information about your company, its products and services, finances, sales, and marketing strategy is a weapon of modern economic warfare. Employees in almost every industry today use messaging platforms like Slack, Microsoft Teams, and Signal to conduct business. But on many of those platforms, messages are "ephemeral": messages can ... If you suspect that you are being sabotaged at work, it's important to recognize the signs early on and take appropriate action to protect yourself and your professional reputation. In this blog, we will explore common signs of workplace sabotage and provide insights on how to create a mindset to navigate these challenging situations. murky world of corporate sabot

In [140]:
docs[100].page_content

'EastEnders \'is set for US remake\'\n\nPlans to create a US soap based on the BBC\'s EastEnders have reportedly been drawn up by the Fox TV network.\n\nEastEnders\' head writer Tony Jordan and music mogul Simon Fuller are involved in the project, according to reports in the Hollywood Reporter trade newspaper. It said scripts have been commissioned for a series about a community of working class people in of Chicago. The original EastEnders was pulled from BBC America last year after it proved a failure in the ratings. US versions of other British hits have proved less successful across the Atlantic.\n\nBBC comedy Coupling was remade with a US cast, but lost its primetime slot on the NBC network due to disappointing ratings. At home, EastEnders has been facing its own ratings battle, recently losing out to rival ITV soap Emmer dale. Primetime soaps on US television have made a recent comeback, following the success of ABC serial Desperate Housewives. The series takes a "darkly comedic"

In [141]:
vectordb.embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: T5EncoderModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False})
  (2): Dense({'in_features': 1024, 'out_features': 768, 'bias': False, 'activation_function': 'torch.nn.modules.linear.Identity'})
  (3): Normalize()
), model_name='sentence-transformers/gtr-t5-large', cache_folder=None, model_kwargs={'device': 'cuda'}, encode_kwargs={'normalize_embeddings': False}, multi_process=False, show_progress=False)